In [ ]:
# Liste des 35 villes à analyser

cities = [
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Ariege",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle"
]

print(f"📍 Nombre de villes à géocoder : {len(cities)}")

📍 Nombre de villes à géocoder : 35


In [2]:
# Créer la fonction de géocodage

import requests
import time

def get_city_coordinates(city_name):
    """
    Récupère les coordonnées GPS d'une ville via l'API Nominatim.
    
    Args:
        city_name (str): Nom de la ville
        
    Returns:
        dict: Dictionnaire contenant city, latitude, longitude, display_name
              Retourne None si la ville n'est pas trouvée
    """
    url = "https://nominatim.openstreetmap.org/search"
    
    params = {
        'q': city_name + ', France',  # Ajouter France pour affiner
        'format': 'json',
        'limit': 1,  # On ne veut que le premier résultat
        'addressdetails': 1
    }
    
    headers = {
        'User-Agent': 'KayakProject/1.0 (Educational Purpose)'  # Obligatoire !
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # Vérifie les erreurs HTTP
        
        data = response.json()
        
        if data and len(data) > 0:
            result = data[0]
            return {
                'city': city_name,
                'latitude': float(result['lat']),
                'longitude': float(result['lon']),
                'display_name': result['display_name']
            }
        else:
            print(f"⚠️ Ville non trouvée : {city_name}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur pour {city_name}: {e}")
        return None

In [ ]:
# Test sur Paris
test_city = get_city_coordinates("Paris")
print(test_city)

{'city': 'Paris', 'latitude': 48.8534951, 'longitude': 2.3483915, 'display_name': 'Paris, Île-de-France, France métropolitaine, France'}


In [4]:
# Boucle pour géocoder les 35 villes

from tqdm import tqdm  # Pour la barre de progression

# Liste pour stocker les résultats
cities_data = []

print("🌍 Démarrage du géocodage...\n")

# Boucle sur chaque ville avec barre de progression
for city in tqdm(cities, desc="Géocodage en cours"):
    # Obtenir les coordonnées
    coords = get_city_coordinates(city)
    
    if coords:
        cities_data.append(coords)
    
    # ⚠️ IMPORTANT : Pause de 1 seconde entre chaque requête
    # Nominatim limite à 1 requête par seconde
    time.sleep(1)

print(f"\n✅ Géocodage terminé !")
print(f"📊 Villes géocodées avec succès : {len(cities_data)} / {len(cities)}")

🌍 Démarrage du géocodage...



Géocodage en cours: 100%|██████████| 35/35 [02:15<00:00,  3.86s/it]


✅ Géocodage terminé !
📊 Villes géocodées avec succès : 35 / 35


In [7]:
import pandas as pd

# Créer le DataFrame
df_cities = pd.DataFrame(cities_data)

# Ajouter un ID unique pour chaque ville
df_cities['city_id'] = range(1, len(df_cities) + 1)

# Réorganiser les colonnes
df_cities = df_cities[['city_id', 'city', 'latitude', 'longitude', 'display_name']]

# Afficher les premières lignes
df_cities.head(10)

,city_id,city,latitude,longitude,display_name
0,1,Mont Saint Michel,48.635954,-1.511460,"Mont Saint-Michel, Le Mont-Saint-Michel, Avran..."
1,2,St Malo,48.649518,-2.026041,"Saint-Malo, Ille-et-Vilaine, Bretagne, France ..."
2,3,Bayeux,49.276462,-0.702474,"Bayeux, Calvados, Normandie, France métropolit..."
3,4,Le Havre,49.493898,0.107973,"Le Havre, Seine-Maritime, Normandie, France mé..."
4,5,Rouen,49.440459,1.093966,"Rouen, Seine-Maritime, Normandie, France métro..."
5,6,Paris,48.853495,2.348391,"Paris, Île-de-France, France métropolitaine, F..."
6,7,Amiens,49.894171,2.295695,"Amiens, Somme, Hauts-de-France, France métropo..."
7,8,Lille,50.636565,3.063528,"Lille, Nord, Hauts-de-France, France métropoli..."
8,9,Strasbourg,48.584614,7.750713,"Strasbourg, Bas-Rhin, Collectivité européenne ..."
9,10,Chateau du Haut Koenigsbourg,48.249411,7.344320,"Château du Haut-Kœnigsbourg, D 159, Château du..."


In [8]:
# Vérifier les données

# Statistiques descriptives
print("\n📊 Statistiques des coordonnées :")
print(df_cities[['latitude', 'longitude']].describe())

# Vérifier s'il manque des villes
print(f"\n🔍 Nombre de villes géocodées : {len(df_cities)}")
print(f"🔍 Villes manquantes : {len(cities) - len(df_cities)}")

# Afficher les villes manquantes si il y en a
if len(df_cities) < len(cities):
    geocoded_cities = df_cities['city'].tolist()
    missing = [c for c in cities if c not in geocoded_cities]
    print(f"⚠️ Villes non trouvées : {missing}")


📊 Statistiques des coordonnées :
        latitude  longitude
count  35.000000  35.000000
mean   45.841184   3.396543
std     2.589584   2.954059
min    42.525050  -2.026041
25%    43.512178   1.380777
50%    45.187560   4.360069
75%    48.417012   5.637707
max    50.636565   7.750713

🔍 Nombre de villes géocodées : 35
🔍 Villes manquantes : 0
